In [1]:
from isochrones.dartmouth import Dartmouth_Isochrone
dar = Dartmouth_Isochrone()

In [2]:
%%file example.obs
name band resolution mag e_mag separation pa relative
twomass J 2.8 10 0.02 0 0 False 
twomass H 2.9 9.6 0.02 0 0 False 
twomass K 3.0 9.4 0.02 0 0 False 
UKIRT J 1.0 11 0.02 0 0 True 
UKIRT J 1.0 14.5 0.02 2. 280 True 
nirc2 K 0.1 0 0 0 0 True 
nirc2 K 0.1 4.0 0.03 2.1 274 True 
nirc2 K 0.1 1.5 0.03 0.3 123 True


Overwriting example.obs


In [3]:
import pandas as pd

df = pd.read_table('example.obs', delim_whitespace=True)#, index_col=[0,1])
df

,name,band,resolution,mag,e_mag,separation,pa,relative
0,twomass,J,2.8,10.0,0.02,0.0,0,False
1,twomass,H,2.9,9.6,0.02,0.0,0,False
2,twomass,K,3.0,9.4,0.02,0.0,0,False
3,UKIRT,J,1.0,11.0,0.02,0.0,0,True
4,UKIRT,J,1.0,14.5,0.02,2.0,280,True
5,nirc2,K,0.1,0.0,0.00,0.0,0,True
6,nirc2,K,0.1,4.0,0.03,2.1,274,True
7,nirc2,K,0.1,1.5,0.03,0.3,123,True


In [4]:
from isochrones.observation import ObservationTree

tree = ObservationTree.from_df(df)
tree.print_ascii()

root
 ╚═ twomass K=(9.4, 0.02) @(0.00, 0)
    ╚═ twomass H=(9.6, 0.02) @(0.00, 0)
       ╚═ twomass J=(10.0, 0.02) @(0.00, 0)
          ╠═ UKIRT J=(11.0, 0.02) @(0.00, 0)
          ║  ╠═ nirc2 K=(0.0, 0.0) @(0.00, 0)
          ║  ╚═ nirc2 K=(1.5, 0.03) @(0.30, 123)
          ╚═ UKIRT J=(14.5, 0.02) @(2.00, 280)
             ╚═ nirc2 K=(4.0, 0.03) @(2.10, 274)


In [5]:
tree.define_models(dar, N=(2,1,1), index=(0,0,1))
tree.print_ascii()

root
 ╚═ twomass K=(9.4, 0.02) @(0.00, 0)
    ╚═ twomass H=(9.6, 0.02) @(0.00, 0)
       ╚═ twomass J=(10.0, 0.02) @(0.00, 0)
          ╠═ UKIRT J=(11.0, 0.02) @(0.00, 0)
          ║  ╠═ nirc2 K=(0.0, 0.0) @(0.00, 0)
          ║  ║  ╠═ 0_0
          ║  ║  ╚═ 0_1
          ║  ╚═ nirc2 K=(1.5, 0.03) @(0.30, 123)
          ║     ╚═ 0_2
          ╚═ UKIRT J=(14.5, 0.02) @(2.00, 280)
             ╚═ nirc2 K=(4.0, 0.03) @(2.10, 274)
                ╚═ 1_0


In [6]:
pars = [1.0,0.9,0.8,9.5,0.0,200,0.1,1.5,9.3,0.1,500,0.2]
pdict = tree.p2pardict(pars)

In [7]:
%timeit tree.lnlike(pars)

The slowest run took 26019.48 times longer than the fastest. This could mean that an intermediate result is being cached 
1 loops, best of 3: 61 µs per loop


In [9]:
%timeit dar.mag['K'](1,9.5,0.0,200,0.2)

10000 loops, best of 3: 84.7 µs per loop


In [10]:
for n in tree:
    if not n.is_leaf:
        print '{}: {} ({})'.format(n.label,[l.label for l in n.leaves],id(n))
        print n.lnlike(pdict)
        print 'reference: {} {}'.format(n.reference,id(n.reference))
        print 'child n_leaves: {}'.format([len(m.leaves) for m in n.children])
        print 'Nstars: {}'.format(n.Nstars)

nirc2 K=(0.0, 0.0) @(0.00, 0): ['0_0', '0_1'] (4513588368)
0
reference: None 4296531008
child n_leaves: [1, 1]
Nstars: {0: 2}
nirc2 K=(1.5, 0.03) @(0.30, 123): ['0_2'] (4513588432)
-19.5741913031
reference: nirc2 K=(0.0, 0.0) @(0.00, 0) 4513588368
child n_leaves: [1]
Nstars: {0: 1}
UKIRT J=(11.0, 0.02) @(0.00, 0): ['0_0', '0_1', '0_2'] (4513588240)
0
reference: None 4296531008
child n_leaves: [2, 1]
Nstars: {0: 3}
nirc2 K=(4.0, 0.03) @(2.10, 274): ['1_0'] (4513588496)
-6546.33795396
reference: nirc2 K=(0.0, 0.0) @(0.00, 0) 4513588368
child n_leaves: [1]
Nstars: {1: 1}
UKIRT J=(14.5, 0.02) @(2.00, 280): ['1_0'] (4513588304)
-235454.425692
reference: UKIRT J=(11.0, 0.02) @(0.00, 0) 4513588240
child n_leaves: [1]
Nstars: {1: 1}
twomass J=(10.0, 0.02) @(0.00, 0): ['0_0', '0_1', '0_2', '1_0'] (4513588176)
-0.0794868533028
reference: None 4296531008
child n_leaves: [3, 1]
Nstars: {0: 3, 1: 1}
twomass H=(9.6, 0.02) @(0.00, 0): ['0_0', '0_1', '0_2', '1_0'] (4513588112)
-208.053788242
reference

AttributeError: 'ObservationTree' object has no attribute 'reference'

In [8]:
l = [1,2,3,4,5]
l.index(10)

ValueError: 10 is not in list